<a href="https://colab.research.google.com/github/JonatanSiracusa/DS-Grupo1/blob/main/Grupo1_Parcial_Bimestre_1_TodosJuntos_version_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio Tipo Parcial

El ministerio de salud de la nación les solicita realizar un programa que les permita visualizar el estado de ciudadanos infectados con COVID-19 en todas sus variantes en un rango de fecha determinado y comparar esos resultados con los niveles de vacunación.
A su vez, se les solicita que realicen una compración con otro(s) país(es). (Seleccionar como máximo 2 adicionales)

Para ello, un analista le provee los siguientes archivos.

[Vacunación Global](https://raw.githubusercontent.com/al34n1x/DataScience/master/100.Examen/2022/country_vaccinations.csv)

[Casos COVID Global](https://raw.githubusercontent.com/al34n1x/DataScience/master/100.Examen/2022/owid-covid-data.csv)


Consideraciones:

* Rango de Fechas desde 01/01/2021 a 15/01/2021
* Importar los archivos y trabajarlos como diccionarios o dataframes. El caracter delimitador de los archivos es el **';'**
* Realizar al menos 4 charts que representen la información solicitada
* Utilizar funciones
* Generar un archivo que posea la siguiente estructura de datos dscriminado por el rango de fechas.
```
ID:{
'país':value,
'fecha':value,
'nuevos_casos':value,
'nuevas_muertes':value,
'total_vacunados':value
}
```
* Documentar cada paso con bloques de texto adicionalmente a notas en el código
* Presentar por pantalla el total acumulado de infectados y de vacunados en el rango de fechas solicitado
* Proveer una breve conclusión como parte del análisis 





## TIPS

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv
file = ('./sample_data/country_vaccinations.csv')
covid_vaccines={}

with open(file, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file,delimiter=';') #Delimitador ;
    line_count = 0
    for row in csv_reader:
      print(row)

In [ ]:
import csv
file = ('./sample_data/owid-covid-data.csv')
#file = ('https://github.com/al34n1x/DataScience/tree/master/100.Examen/2022/owid-covid-data.csv')
covid_cases={}

with open(file, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file,delimiter=';')
    line_count = 0
    for row in csv_reader:
      print(row)

In [ ]:
var = '1.0'

split = var.split('.')[0] # Dividir cadena de caracteres para eliminar los caracteres detras del .

print(var)
print(int(split))

---


## Carga de datos

In [11]:
import pandas as pd

#Obtenemos los datos del archivo de vacunacion
vacunacion = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/100.Examen/2022/country_vaccinations.csv'
  , delimiter=';')

#vacunacion = vacunacion.fillna ({"daily_vaccinations": 0})

#Pasamos a date la columna de fechas, para luego poder filtrar
vacunacion['date'] = pd.to_datetime(vacunacion['date'], dayfirst=True)

#Filtramos los datos por las fechas solicitadas. Se agregan, a su vez, otros filtros.
mask = (vacunacion['date'] >= '2021-01-01') & (vacunacion['date'] <= '2021-01-15')
vacunacion_filt=vacunacion.loc[mask]

mask2 = (vacunacion['date'] >= '2021-01-01') & (vacunacion['date'] <= '2021-01-15') & ((vacunacion['iso_code'] == 'ARG') | (vacunacion['iso_code'] == 'CHN') | (vacunacion['iso_code'] == 'USA'))
vacunaciones_diarias = vacunacion.loc[mask2]


In [12]:
#Obtenemos los datos del archivo de informacion global
covid_global = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/100.Examen/2022/owid-covid-data.csv'
  , delimiter=',')


#Pasar a date la columna de fechas, para luego poder filtrar
covid_global['date'] = pd.to_datetime(covid_global['date'], dayfirst=True)


#Filtramos los datos por las fechas solicitadas. Se agregan, a su vez, otros filtros.
mask = (covid_global['date'] >= '2021-01-01') & (covid_global['date'] <= '2021-01-15')
covid_global_filt=covid_global.loc[mask]

mask2 = (covid_global['date'] >= '2021-01-01') & (covid_global['date'] <= '2021-01-15') & ((covid_global['iso_code'] == 'ARG') | (covid_global['iso_code'] == 'CHN') | (covid_global['iso_code'] == 'USA'))
covid_globales = covid_global.loc[mask2]


#Filtramos las columnas que nos pueden interesar para el archivo de datos globales
covid_global_col=pd.DataFrame(covid_global_filt, columns=['iso_code','continent','location','date','total_cases','new_cases','new_deaths','total_deaths'])

covid_datos_globales = pd.DataFrame(covid_globales, columns=['iso_code','continent','location','date','total_cases','new_cases','new_deaths','total_deaths'])


## Limpieza de datos

In [ ]:
# Ingrese código aquí

In [ ]:
# Ingrese código aquí

In [ ]:
# Ingrese código aquí

## Transformación de datos

Combinamos los DataFrames anteriormente generados.

In [13]:
#Unimos la ionformación de ambos origenes de datos, por país y fecha
outer_merged = pd.merge(covid_global_col, vacunacion_filt, how="outer", on=["iso_code", "date"])

dfs_merged = pd.merge(covid_datos_globales, vacunaciones_diarias, how="outer", on=["iso_code", "date"])
df_datos = dfs_merged


#Completamos los NaN de vacunacion con ceros
covid_merge = outer_merged.fillna ({"daily_vaccinations": 0})


#Filtramos por los países que vamos a trabajar
mask = (covid_merge['iso_code'] == 'ARG') | (covid_merge['iso_code'] == 'CHN') | (covid_merge['iso_code'] == 'USA')
covid_merge_filt=covid_merge.loc[mask]

Recorremos el DataFrame para crear una cadena de caracteres similar a un archivo de formato JSON, con la información solicitada.

In [14]:
info_covid_json = "["

for i in df_datos.index:
  pais = df_datos['location'][i]
  fecha = str(df_datos['date'][i])[0:10]
  nuevos_casos = str(df_datos['new_cases'][i]).split(".")[0]
  nuevas_muertes = str(df_datos['new_deaths'][i]).split(".")[0]
  total_vacunados = str(df_datos['daily_vaccinations'][i]).split(".")[0]

  info_covid_json = info_covid_json + str(i) + ": {"
  info_covid_json = info_covid_json + "'pais': " + pais + ", "
  info_covid_json = info_covid_json + "'fecha': " + fecha + ", "
  info_covid_json = info_covid_json + "'nuevos_casos': " + nuevos_casos + ", "
  info_covid_json = info_covid_json + "'nuevas_muertes': " + nuevas_muertes + ", " 
  info_covid_json = info_covid_json + "'total_vacunados': " + total_vacunados
  info_covid_json = info_covid_json + "}, "


info_covid_json = info_covid_json[0:-2]
info_covid_json = info_covid_json + "]"

In [ ]:
# Ingrese código aquí

## Análisis y Visualizaciones

*Ingrese su análisis aquí*


In [15]:
#Crear funcion que plotea
def Chart1(datos):
  import matplotlib.pyplot as plt
  import pandas as pd 
  import numpy as np

  #Trabajar sobre una copia
  datos=covid_merge_filt.copy()

  #Obtener Países
  paises=pd.Series(datos['iso_code'].drop_duplicates())
  
  #Por cada país genero la información
  for p in paises:

    #fig= plt.figure(constrained_layout = True,facecolor='#EBEBEB', edgecolor='black')
    fig= plt.figure(figsize=(5,3),facecolor='#EBEBEB', edgecolor='black')

    #gs = fig.add_gridspec(4, 7)
    ax = fig.add_axes([0,0,1,1],facecolor='lightsteelblue') 

    #Obtener datos de ese país
    pais=datos[datos['iso_code']==p]
    #Obtener las fechas
    fechas=pd.to_datetime(pais['date'].drop_duplicates()).astype(str).tolist()
    #Obtener los vacunados
    vacunados=pd.Series(pais['daily_vaccinations'])
    #Obtener los infectados
    infectados=pd.Series(pais['new_cases'])
    #Obtener la cantidad de bins(fechas)
    cantFechas=np.arange(len(fechas))
  
    #Crear barra de vacunados
    ax.bar( x=cantFechas-0.2
          ,height=vacunados
          ,width=0.4           
          ,edgecolor="black"  
          ,linewidth=0.8
          ,color='#36648B'          
          )
    #Crear barra de infectados
    ax.bar( cantFechas+0.2
        ,height=infectados  
        ,width=0.4         
        ,edgecolor="black"  
        ,linewidth=0.8
        ,color='#36998B'          
          )
    #Crear leyenda
    ax.legend(['Vacunados', 'Infectados'], loc='upper right')

    #Aplicar parametros a los ticks de los ejes
    plt.yticks(fontsize=16)
    plt.xticks(ticks=cantFechas, labels=fechas, rotation=90, fontsize=12)

    #Aplicar etiquetas
    plt.ylabel('Cantidad', fontdict = {'fontsize':20, 'color':'tab:blue'})  
    plt.xlabel('Fecha', fontdict = {'fontsize':20, 'color':'tab:blue'})
    plt.title( p + " Covid19", loc = "center", fontdict = {'Weight':'bold', 'fontsize':20, 'color':'tab:blue'})
   
    #Dibujar
    plt.show()

In [16]:
#Porbar funcion que plotea
#covid_merge_filt
#Chart1(covid_merge_filt)

In [ ]:
# Ingrese código aquí

Gráfico 1 – Comparación de infectados en países sudamericanos.

Ver los totales de vacunados para el rango de fechas y sacar conclusiones de los nuevos contagios y las nuevas muertes.

Gráfico 2 – Comparación Argentina vs Estados Unidos vs China.

Ver total de contagios y total de vacunados dentro del rango de fechas comparando Argentina contra los países más fuertes del mundo.

Gráfico 3 – Comparación de países en estaciones climáticas diferentes.

Ver nuevos contagiados para el rango de fechas y sacar conclusiones de los nuevos contagios con la premisa de que en países donde están transitando el invierno, la gente es más propensa a contagiarse.

Gráfico 4 – Comparación de países representativos por continente.

Seleccionando un país representativo por continente, hacer una comparación de total de vacunados para poder sacar conclusiones del nivel socio-económico influye en la vacunación de la población.

Imprimimos por pantalla el total acumulado de infectados y de vacunados en el rango de fechas solicitado.

In [17]:
total_acum_infectados = 0
total_acum_vacunados = 0

for i in df_datos.index:
  nuevos_casos = (df_datos['new_cases'][i])
  total_vacunados = (df_datos['daily_vaccinations'][i])

  total_acum_infectados += nuevos_casos
  total_acum_vacunados += total_vacunados

print("Información solicitados: \n")
print("Total acumulado de infectados = ", total_acum_infectados) 
print("Total acumulado de vacunados = ", total_acum_vacunados)

Información solicitados: 

Total acumulado de infectados =  3693493.0
Total acumulado de vacunados =  13521439.0


## Conclusión

En base a los datos analizados, llegamos a las siguientes conclusiones.

Deducimos que la cantidad de infectados por día y los vacunados por día, en Argentina, no presentan una correlación, ni una relación causa-efecto directa.

En relación a los datos provenientes de China lo más notable es que no existen valores de "nuevas muertes" durante una fracción de tiempo, lo cual posibilita inferir que los datos muy probablemente no reflejen la realidad. Este dato con valores inexistentes imposibilita realizar algún tipo de análisis de correlación entre la cantidad de vacunados por día y la cantidad de nuevas muertes.

Comparando la cantidad de vacunaciones diarias de Estados Unidos y China, es claramente observable que Estados Unidos tuvo un ratio de vacunación diaria per cápita mucho mayor al de China.


In [ ]:
# Ingrese código aquí

## Almacenamiento de resultados

Creamos el archivo JSON solicitado conteniendo el diccionario generado.

In [18]:
import json

filename = './sample_data/info_covid_20210101-20210115.json'

with open (filename, 'w') as archivo:
  json.dump(info_covid_json, archivo)

print('\nSe creo un archivo en formato JSON con la información solicitada.\n\nEl nombre del archivo es "info_covid_20210101-20210115.json" y se encuentra en "./sample_data" del presente notebook.') 


Se creo un archivo en formato JSON con la información solicitada.

El nombre del archivo es "info_covid_20210101-20210115.json" y se encuentra en "./sample_data" del presente notebook.
